This training run is on the initial dataset given by Davut, (named: airfoil_9k_subset2k.csv). The model is being run using an NIF model.

In [1]:
%load_ext autoreload
%autoreload 2

# Define autroreload so that it doesn't cause pain in the ass when we change the functions and run this notebook

In [2]:
# Import all the models, training functions, manipulators here

# Define the relative paths, append it to the system path
import sys
from pathlib import Path
project_root = Path.cwd().resolve().parents[3]
sys.path.append(str(project_root))

# Import shenanigans
from defs.helper_functions.training_functions import *
from defs.helper_functions.data_manipulation_functions import *
from defs.helper_functions.create_model_and_optimizer_functions import *

# Time, to precisely: time
import time

In [3]:
# Set device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")  # Force CPU for testing purposes
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Import and manipulate all the dataset required

df = pd.read_csv(rf"{project_root}\datasets\airfoil_9k_subset2k.csv") # read csv
df = datareform_tuple_coordinates(df=df, which_coord="y") # take in the y vals
shuffled_df = df.sample(frac=1, random_state=69).reset_index(drop=True) # shuffle the dataframe
input_tensor, target_tensor = seperate_transform(df=df, target_coord=18) # get the target and input tensors
input_tensor = input_tensor.to(device)
target_tensor = target_tensor.to(device)
del df # Clear memory

In [5]:
"""
Trial 1, define configs for Trial 1 here, set device
"""

import torch.nn as nn

cfg_shape_net = {
    'input_dim': 15,
    'output_dim': 3,
    'hidden_units': [8,16,32,16,8],
    'shape_activation': nn.Tanh
}

cfg_param_net = {
    'input_dim': 3,
    'hidden_units': [8,16,16,8],
    'param_activation': nn.Tanh
}

NIF_model, NIF_optimizer = create_NIF_Pointwise_and_optimizer(cfg_shape_net=cfg_shape_net, cfg_param_net=cfg_param_net, lr=0.00005)
NIF_model = NIF_model.to(device)

In [6]:
# Show the model

from torchviz import make_dot

# Create and move inputs to device
show_tensor = torch.randn(5, cfg_shape_net['input_dim'] + cfg_param_net['input_dim']).to(device)

# Forward pass
output = NIF_model(show_tensor)

make_dot(output, params=dict(NIF_model.named_parameters())).render("model_graph", format="png")

'model_graph.png'

In [7]:
# Now that we have imported data, defined model, and seen what its like, we'll train here


def mse_loss(predictions, targets): return torch.mean((predictions - targets) ** 2)
torch.cuda.synchronize()
start = time.perf_counter()
epoch_train = train_one_epoch(model= NIF_model, optimizer= NIF_optimizer, criterion=mse_loss, inputs= input_tensor, targets= target_tensor, batch_size= 2, device= device, lambda_reg= 0)
torch.cuda.synchronize()
end = time.perf_counter()
print(f"Training took {end - start} seconds")

0.23663949966430664
Rows: 0 to 1 calculating
0.09319370985031128
Rows: 2 to 3 calculating
0.12842333316802979
Rows: 4 to 5 calculating
0.31997150182724
Rows: 6 to 7 calculating
0.48266133666038513
Rows: 8 to 9 calculating
0.543220579624176
Rows: 10 to 11 calculating
0.28540706634521484
Rows: 12 to 13 calculating
0.20034855604171753
Rows: 14 to 15 calculating
0.1778108924627304
Rows: 16 to 17 calculating
0.4900748133659363
Rows: 18 to 19 calculating
0.4734259843826294
Rows: 20 to 21 calculating
0.30780029296875
Rows: 22 to 23 calculating
0.4721865653991699
Rows: 24 to 25 calculating
0.11290361732244492
Rows: 26 to 27 calculating
0.3006478548049927
Rows: 28 to 29 calculating
0.7092692852020264
Rows: 30 to 31 calculating
0.39503857493400574
Rows: 32 to 33 calculating
0.48608309030532837
Rows: 34 to 35 calculating
0.2143758237361908
Rows: 36 to 37 calculating
0.3990609347820282
Rows: 38 to 39 calculating
0.637121319770813
Rows: 40 to 41 calculating
0.5283782482147217
Rows: 42 to 43 calcula

In [8]:
# Brief trial to ensure the model updates, we will calculate the MSE for the first row of data

start_idx = 2

trial_input = input_tensor[start_idx:start_idx+1].to(device)

result = NIF_model(trial_input)

trial_target = target_tensor[start_idx:start_idx+1].to(device)
trial_mse = mse_loss(result, trial_target).item()
print("Trial MSE for the first input-target pair:")

print(trial_mse)

Trial MSE for the first input-target pair:
0.038547080010175705


In [9]:

"""

Findings:

1- The model trains incredibly well only for a single instance, however, I think it cannot generalize among different airfoils.A solution to this naturally is to feed it more data
2- The model seems to be saturated, at least for now, with the layer configs: [8,16,32,16,8] for shape net and [8,16,16,8] for param net.
3- The training is relatively fast, although for some reason gpu and cpu training takes similar amounts of time. A reason to this may be that I may not be throwing everything to the GPU, I throw the tensors and the model.

""" 

'\n\nFindings:\n\n1- The model trains incredibly well only for a single instance, however, I think it cannot generalize among different airfoils.A solution to this naturally is to feed it more data\n2- The model seems to be saturated, at least for now, with the layer configs: [8,16,32,16,8] for shape net and [8,16,16,8] for param net.\n3- The training is relatively fast, although for some reason gpu and cpu training takes similar amounts of time. A reason to this may be that I may not be throwing everything to the GPU, I throw the tensors and the model.\n\n'